In [1]:
# Flask에서 list를 return 할 수 없으므로 jsonify import (show member list)
from flask import Flask, request, jsonify

In [2]:
class Messenger:
    def __init__(self):
        self.message_box = []
        self.member_list = []
        # 유저의 비밀번호를 딕셔너리 형태로 선언
        self.password_box = {}
        
    def empty_message_box(self):
        self.message_box = []
        
    def write_message(self, name, content, time):
        if self.is_existing_member(name):
            # 시간
            self.message_box.append({"name":name,"content":content, "time":time})
        
    def get_message_box(self):
        return self.message_box
    
    def delete_all_messages_by_name(self, name):
        if self.is_existing_member(name):
            for i in range(len(self.message_box), 0, -1):
                if self.message_box[i-1]["name"] == name:
                    del self.message_box[i-1]

    # join시 비밀번호 입력 추가    
    def join(self, name, password):
        if not self.is_existing_member(name):
            self.member_list.append(name)
            # join시 password_box에 데이터 추가    
            self.password_box[name] = password
        
    def is_existing_member(self, name):
        return (name in self.member_list)

    # 비밀번호를 체크하는 함수
    def check_password(self, name, password):
        # 딕셔너리의 value값을 pwd라고 선언
        pwd = self.password_box.get(name) 
        # pwd가 password와 같다면 True return
        if pwd == password: 
            return True
        else:
            return False
    
    def withdraw(self, name):
        if self.is_existing_member(name):
            self.delete_all_messages_by_name(name)
            self.member_list.remove(name)
            
    def get_messages_by_text(self):
        messages = ""
        for message in self.message_box:
            messages = messages + "[%s]%s: %s \n"%(message["time"], message["name"], message["content"])
        return jsonify(messages)

In [12]:
app = Flask(__name__)
global messenger

@app.route('/join', methods=["GET"])
def join():
    name = request.args.get("name")
    password = request.args.get("password")
    if not messenger.is_existing_member(name):
        # join시 비밀번호 입력 추가
        messenger.join(name, password)
        return "SUCCESS"
    else:
        return "FAIL"

@app.route('/write', methods=["GET"])
def write():
    name = request.args.get("name")
    password = request.args.get("password")
    content = request.args.get("content")
    # 시간
    time = request.args.get("time")
    if messenger.is_existing_member(name):
        # 비밀번호가 맞는지 판단
        if messenger.check_password(name, password) == True:
            # 비밀번호가 맞다면 메시지 작성
            messenger.write_message(name, content, time)
            return "SUCCESS"
        else:
            # 비밀번호가 틀리면 리턴
            return "WRONG PASSWORD!"
        return "SUCCESS"
    else:
        return "FAIL"    
    
@app.route('/update', methods=["GET"])
def update():
    name = request.args.get("name")
    if messenger.is_existing_member(name):
        return messenger.get_messages_by_text()
    else:
        return "FAIL"
    
@app.route('/withdraw', methods=["GET"])
def withdraw():
    name = request.args.get("name")
    if messenger.is_existing_member(name):
        messenger.withdraw(name)
        return "SUCCESS"
    else:
        return "FAIL"

# 멤버리스트를 보여주기
@app.route('/member_list', methods=["GET"])
def member_list():
    name = request.args.get("name")
    if messenger.is_existing_member(name):
        # 리스트를 json파일 형식으로 변환 후 리턴(Flask에서는 리스트 return 불가)
        return jsonify(messenger.member_list)
    else:
        return "FALSE"

if __name__ == "__main__":
    messenger = Messenger()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
